In [1]:
import pandas as pd
import numpy as np

# Load the cleanest available file
try:
    df = pd.read_csv("../data/train_snapshot_after_eda.csv")
    print("Loaded snapshot. Shape:", df.shape)
except FileNotFoundError:
    df = pd.read_csv("../data/train.csv")
    print("Loaded raw train.csv. Shape:", df.shape)

df.head()


Loaded snapshot. Shape: (1460, 81)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [2]:
# Drop row identifier
if "Id" in df.columns:
    df = df.drop(columns=["Id"])

# Optional: remove 2 extreme outliers commonly noted in this dataset
if {"GrLivArea","SalePrice"}.issubset(df.columns):
    before = df.shape[0]
    df = df[~((df["GrLivArea"] > 4500) & (df["SalePrice"] < 300000))].copy()
    print(f"Outlier trim: {before - df.shape[0]} rows removed")

print("Shape:", df.shape)


Outlier trim: 2 rows removed
Shape: (1458, 80)


In [3]:
missing_ratio = df.isna().sum().div(len(df))
to_drop = missing_ratio[missing_ratio > 0.40].index.tolist()
df = df.drop(columns=to_drop)
print("Dropped (high-missing):", to_drop)
print("Remaining shape:", df.shape)


Dropped (high-missing): ['Alley', 'MasVnrType', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature']
Remaining shape: (1458, 74)


In [4]:
# Helper to fill if column exists
def fill_col(col, func):
    if col in df.columns:
        df[col] = func(df[col])

# --- Smarter imputations for known fields ---
# LotFrontage: median by Neighborhood
if {"LotFrontage","Neighborhood"}.issubset(df.columns):
    df["LotFrontage"] = df.groupby("Neighborhood")["LotFrontage"].transform(
        lambda s: s.fillna(s.median())
    )

# GarageYrBlt: use YearBuilt when NaN
if {"GarageYrBlt","YearBuilt"}.issubset(df.columns):
    df["GarageYrBlt"] = df["GarageYrBlt"].fillna(df["YearBuilt"])

# MasVnrType/Area: None and 0
fill_col("MasVnrType", lambda s: s.fillna("None"))
fill_col("MasVnrArea", lambda s: s.fillna(0))

# Many categorical NAs in this dataset mean "None"
none_cats = [
    "Alley","BsmtQual","BsmtCond","BsmtExposure","BsmtFinType1","BsmtFinType2",
    "FireplaceQu","GarageType","GarageFinish","GarageQual","GarageCond",
    "PoolQC","Fence","MiscFeature"
]
for c in none_cats:
    if c in df.columns:
        df[c] = df[c].fillna("None")

# Remaining: numeric -> median, categorical -> mode
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = df.select_dtypes(include=["object"]).columns.tolist()

for c in num_cols:
    if df[c].isna().any():
        df[c] = df[c].fillna(df[c].median())

for c in cat_cols:
    if df[c].isna().any():
        df[c] = df[c].fillna(df[c].mode()[0])

print("Total remaining missing:", int(df.isna().sum().sum()))


Total remaining missing: 0


In [5]:
# Step X: Data Cleaning — check for duplicates, constants, and inconsistent categories

# 1. Check for duplicate rows
dupes = df.duplicated().sum()
print(f"Duplicate rows found: {dupes}")
if dupes > 0:
    df = df.drop_duplicates()
    print(f"Duplicates dropped. New shape: {df.shape}")

# 2. Check for constant columns (no variance)
const_cols = [c for c in df.columns if df[c].nunique() == 1]
if const_cols:
    print("Constant columns:", const_cols)
    df = df.drop(columns=const_cols)
    print(f"Removed constant columns. New shape: {df.shape}")
else:
    print("No constant columns found.")

# 3. Spot check small categorical columns for inconsistent values
for c in df.select_dtypes(include='object').columns:
    if df[c].nunique() < 15:  # small categories
        print(f"\n{c} unique values:\n", df[c].unique())


Duplicate rows found: 0
No constant columns found.

MSZoning unique values:
 ['RL' 'RM' 'C (all)' 'FV' 'RH']

Street unique values:
 ['Pave' 'Grvl']

LotShape unique values:
 ['Reg' 'IR1' 'IR2' 'IR3']

LandContour unique values:
 ['Lvl' 'Bnk' 'Low' 'HLS']

Utilities unique values:
 ['AllPub' 'NoSeWa']

LotConfig unique values:
 ['Inside' 'FR2' 'Corner' 'CulDSac' 'FR3']

LandSlope unique values:
 ['Gtl' 'Mod' 'Sev']

Condition1 unique values:
 ['Norm' 'Feedr' 'PosN' 'Artery' 'RRAe' 'RRNn' 'RRAn' 'PosA' 'RRNe']

Condition2 unique values:
 ['Norm' 'Artery' 'RRNn' 'Feedr' 'PosA' 'PosN' 'RRAn' 'RRAe']

BldgType unique values:
 ['1Fam' '2fmCon' 'Duplex' 'TwnhsE' 'Twnhs']

HouseStyle unique values:
 ['2Story' '1Story' '1.5Fin' '1.5Unf' 'SFoyer' 'SLvl' '2.5Unf' '2.5Fin']

RoofStyle unique values:
 ['Gable' 'Hip' 'Gambrel' 'Mansard' 'Flat' 'Shed']

RoofMatl unique values:
 ['CompShg' 'WdShngl' 'Metal' 'WdShake' 'Membran' 'Tar&Grv' 'Roll']

ExterQual unique values:
 ['Gd' 'TA' 'Ex' 'Fa']

ExterC

In [6]:
def add_feature(col, series):
    if col not in df.columns:
        df[col] = series

# Ages
if {"YrSold","YearBuilt"}.issubset(df.columns):
    add_feature("HouseAge", df["YrSold"] - df["YearBuilt"])
if {"YrSold","YearRemodAdd"}.issubset(df.columns):
    add_feature("RemodAge", df["YrSold"] - df["YearRemodAdd"])

# Bathrooms (weighted)
parts = ["FullBath","HalfBath","BsmtFullBath","BsmtHalfBath"]
if all(c in df.columns for c in parts):
    add_feature("TotalBathrooms", df["FullBath"] + 0.5*df["HalfBath"]
                + df["BsmtFullBath"] + 0.5*df["BsmtHalfBath"])

# Porches total
porch_cols = ["OpenPorchSF","EnclosedPorch","3SsnPorch","ScreenPorch"]
if all(c in df.columns for c in porch_cols):
    add_feature("TotalPorchSF", df[porch_cols].sum(axis=1))

print("Engineered columns added. Shape:", df.shape)


Engineered columns added. Shape: (1458, 78)


In [7]:
df_encoded = pd.get_dummies(df, drop_first=True)
print("After encoding:", df_encoded.shape)


After encoding: (1458, 242)


In [8]:
assert "SalePrice" in df_encoded.columns, "SalePrice not found after encoding."

y = np.log1p(df_encoded["SalePrice"])   # log target
X = df_encoded.drop(columns=["SalePrice"])

X.to_csv("../data/X_preprocessed.csv", index=False)
y.to_csv("../data/y_preprocessed.csv", index=False)
print("Saved X_preprocessed.csv and y_preprocessed.csv")
print("X shape:", X.shape)


Saved X_preprocessed.csv and y_preprocessed.csv
X shape: (1458, 241)
